## Import Libraries 

In [1]:
import pandas as pd
import requests
import zipfile
from io import BytesIO

## Set Display Options

In [ ]:
pd.options.display.width = 0

## Download and Extract Data

In [ ]:
zip_url = "https://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_Livestock_E_All_Data.zip"

# Name of the CSV file we want to extract from the zip file
csv_filename_without_extension = "Production_Crops_Livestock_E_All_Data_NOFLAG"

# HTTP GET request to download the zip file
response = requests.get(zip_url)
response.raise_for_status()

# Extract the specific CSV file from the zip file
with zipfile.ZipFile(BytesIO(response.content)) as zip_archive:
    # Find the filename in the archive
    for filename in zip_archive.namelist():
        if csv_filename_without_extension in filename:
            csv_filename = filename
            break
    else:
        raise ValueError(f"CSV file '{csv_filename_without_extension}' not found in the zip archive")

    with zip_archive.open(csv_filename) as csv_file:
        # Read the content of the CSV file into a DataFrame
        detected_encoding = "ISO-8859-1"  
        df = pd.read_csv(csv_file, encoding=detected_encoding)

## Exclude Years Before 2000

In [ ]:
 #Getting the column names, so I can exclude years before 2000
column_names = df.columns

# List of years to be excluded
excluded_years = [f'Y{i}' for i in range(1961, 2000)]

# Drop excluded years
df = df.drop(columns=excluded_years)

## Exclude Regions and Former Countries 

In [ ]:
# Get the unique values in the "Area" column to see what countries I need to exclude
countries_list = df['Area'].unique()

# Print the list of countries
for country in countries_list:
    print(country)
    
# Making the exclusion list for Area
regions_and_old_countries = ['Belgium-Luxembourg', 'China', 'Czechoslovakia','USSR', 'Yugoslav SFR', 'Serbia and Montenegro',
                             'World','Africa', 'Eastern Africa','Middle Africa', 'Northern Africa', 'Southern Africa',
                             'Western Africa', 'Americas', 'Northern America', 'Central America', 'Caribbean', 'South America',
                             'Asia','Central Asia', 'Eastern Asia', 'Southern Asia', 'South-eastern Asia', 'Western Asia',
                             'Europe', 'Eastern Europe', 'Northern Europe','Southern Europe', 'Western Europe','Oceania',
                             'Australia and New Zealand','Melanesia', 'Micronesia', 'Polynesia', 'European Union (27)', 
                             'Least Developed Countries', 'Land Locked Developing Countries','Small Island Developing States',
                             'Low Income Food Deficit Countries','Net Food Importing Developing Countries']
# Filtering conditions
conditions = (
    (df['Element'] == 'Production') &
    (df['Unit'] == 'tonnes') &
    (~df['Area'].isin(regions_and_old_countries)))

# Apply the conditions
df = df[conditions]

## Melt DataFrame 

In [ ]:
# Melt the DataFrame from wide to long format
years = [f'Y{i}' for i in range(2000, 2022)]
df = pd.melt(df, id_vars=['Area Code', 'Area Code (M49)', 'Area', 'Item Code', 'Item',
                          'Element Code', 'Element', 'Unit'], value_vars=years)

## Consolidate Data for Sudan and Former Sudan

In [ ]:
# Consolidate the data for Sudan and Former Sudan
delete_years_sudan = [f'Y{i}' for i in range(2000, 2012)]
delete_years_former_sudan = [f'Y{i}' for i in range(2012, 2022)]

condition1 = ((df['Area'] == 'Sudan') & (df['variable'].isin(delete_years_sudan)))
condition2 = ((df['Area'] == 'Sudan (former)') & (df['variable'].isin(delete_years_former_sudan)))

combined_condition = condition1 | condition2

rowstodelete = df[combined_condition]

df = df[~combined_condition]

df.replace("Sudan (former)", "Sudan", inplace=True)

## Replace Diacritics

In [ ]:
# Replace diacritics
replace_dict = {
    "Côte d'Ivoire": "Cote d'Ivoire",
    "Réunion": "Reunion",
    "Türkiye": "Turkiye"
}

df.replace(replace_dict, inplace=True)

## Final Edits 

In [ ]:
# Final edits
df.rename(columns={'variable': 'Year'}, inplace=True)
df['Year'] = df['Year'].str.replace('Y', '')

## Print the DataFrame in CSV

In [ ]:
# Export the DataFrame to a new CSV file named "Crop Tableau Data".csv"
df.to_csv('Crop_for_Tableau.csv', index=False)

print(df)